In [ ]:
import math
import agentpy as ap
import numpy as np
# Visualization
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import IPython
from IPython.display import display, HTML

In [ ]:
class RandomAgent(ap.Agent):
    """ Se mueve a celdas al azar """
    
    def setup(self):
        self.position = (0,0)  # Posición actual del agente (x, y)
        self.intention = None  # Intención actual (clean, move o wait)
        self.new_position = None  # Objetivo al moverse (x, y)
        self.is_dirty = False
        
    def see(self):
        """ Percibe su entorno """
        x, y = self.position
        self.is_dirty = self.model.is_dirty[x][y]
        self.neighbors = self.model.get_neighbors(x, y) 
    
    def next(self):
        """ Decide la próxima acción según lo que percibe """
        self.intention = "wait" # por defecto espera
        if self.is_dirty:
            self.intention = "clean"
        else:
            self.new_position = self.model.random.choice(self.neighbors)
            if self.model.is_valid_position(self.new_position):
                self.intention = "move"
                
    def action(self):
        """ Ejecuta la accion """
        x, y = self.position
        if self.intention == "clean":
            self.model.clean_cell(x, y)
        elif self.intention == "move":
            self.position = self.new_position
            
    def move(self):
        if not self.model.is_board_clean:
            self.model.total_moves += 1     
        self.see()
        self.next()
        self.action()

        

In [ ]:
class SergioAgent(ap.Agent):
    """ Empieza con una estrategia de llenado en columnas, seguido de busqueda por barrido """
    
    def setup(self):
        self.position = self.p.initial_position  # Posición actual del agente (x, y)
        self.intention = None  # Intención actual (clean, move o wait)
        self.new_position = None  # Objetivo al moverse (x, y)
        self.is_dirty = False
        self.x_dir = 1
        self.y_dir = 1
        self.is_sharing_space = False
        self.changed_dir = 0
        self.delay = 1
        self.state_utility = 0

        
    def see(self):
        """ Percibe su entorno """
        x, y = self.position
        self.is_dirty = self.model.is_dirty[x][y] 
        self.state_utility = 0 # inicialmente es 0. Si logra limpiar es 1

        self.is_sharing_space = 0
        for agent in self.model.all_agents:
            if agent.position == self.position:
                self.is_sharing_space += 1
                if self.is_sharing_space > 1:
                    break
        
        # Removing itself!
        self.is_sharing_space -= 1
        
        x_max = len(self.model.is_dirty) - 1
        y_max = len(self.model.is_dirty[0]) - 1
        if x == 0 and x == x_max:
            self.x_dir = 0
        elif x == 0 and self.x_dir != 1: 
            self.x_dir = 1
            # raise RuntimeError
        elif x == x_max:
            self.x_dir = -1
        
        if y == 0 and y == y_max:
            self.y_dir = 0
        elif y == 0 and self.y_dir != 1 and self.delay <= 0:
            self.changed_dir = 1
            self.y_dir = 1
        elif y == y_max and self.y_dir != -1 and self.delay <= 0:
            self.changed_dir = 1
            self.y_dir = -1
        
        # Prevents double change in dir, just after moving to next column
        self.delay -= 1
            
            
    
    def next(self):
        """ Decide la próxima acción según lo que percibe """
        self.intention = "wait" # por defecto espera
        if self.is_dirty:
            self.delay = 1
            self.intention = "clean"
            return  
        
        x, y = self.position
        if self.changed_dir:
            self.new_position = (x + self.x_dir, y)
            self.changed_dir = 0
            self.delay = 1
        elif self.is_sharing_space:
            # algunos sigue barriendo vertical, otros se mueve horizontal
            possible_moves = [(x + self.x_dir, y), (x, y + self.y_dir)]
            index = self.model.nprandom.choice([0, 1], p=[0.95, 0.05])
            if index == 0:
                self.delay = 1
            self.new_position = possible_moves[index]
        else: 
            self.new_position = (x, y + self.y_dir)
        
        if self.model.is_valid_position(self.new_position):
            self.intention = "move"
                
    def action(self):
        """ Ejecuta la accion """
        x, y = self.position
        if self.intention == "clean":
            self.model.clean_cell(x, y)
            self.state_utility = 1 # Ese estado tiene utilidad debido a limpiar
        elif self.intention == "move":
            self.position = self.new_position
            
    def move(self):
        if not self.model.is_board_clean:
            self.model.total_moves += 1     
        self.see()
        self.next()
        self.action()


In [ ]:
class RodrigoAgent(ap.Agent):
    def setup(self):
        """
        Agente que asigna un numero (calor) a cada uno de los vecinos para escoger a cual moverse
        
        Esto usando una formula que mientras mas cerca de ti hayan celdas sucias te da mas calor
        y mientras mas agentes haya cerca de esa celda le da menos calor
        
        esto para que las celdas con muchas celdas sucias cerca y pocos agentes cerca sean priorizadas
        
        asignamos probabilidades en base a calor y elegimos a donde nos movemos
        """
        self.position = (0, 0)  # Posicion inicial (x, y)
        self.intention = None  # Intencion actual (clean, move, wait)
        self.new_position = None  # Objetivo al moverse (x, y)
        
        # Pesos escogidos para las variables que influyen al calor de una celula
        self.w1 = 1 
        self.w2 = 1
        self.w3 = -2
        self.w4 = -1
        
        self.heatmap = {}  # Aqui guardamos el mapa de calor de los vecinos de nuestra celda
        self.distancesDirty = {}  # Guarda las distancias a celdas sucias
        self.distancesAgent = {}  # Guarda las distancias a otros agentes
        
        # Al principio contamos todas las celdas como sucias
        self.dirtyCells = [(row, col) for row in range(self.model.p.n) for col in range(self.model.p.m)]
        self.run_utility = 0
        self.state_utility = 0
        
    # Cantidad de celdas limpias entre sucias
    def utility(self):
        return self.model.cleaned_cells / self.model.dirty_cells * 100
        
    def calculate_distance(self, pos1, pos2):
        """
        Calcula distancia euclidiana entre 2 puntos (pos1 y pos2).
        """
        x1, y1 = pos1
        x2, y2 = pos2
        return (math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2) + 1)

    def see(self):
        """ Percibe su entorno """
        x, y = self.position
        neighbors = self.model.get_neighbors(x, y)  # Agarra todas las celdas vecinas
        valid_neighbors = [(nx, ny) for nx, ny in neighbors if self.model.is_valid_position((nx, ny))]
        
        self.distancesDirty.clear()  
        self.distancesAgent.clear()  

        # Observa cuales son las celdas suctias
        self.dirtyCells = [(cx, cy) for cx, cy in self.dirtyCells if self.model.is_dirty[cx][cy]]

        # recopila las distancias de sus vecinos a las celdas sucias
        for nx, ny in valid_neighbors:
            # Inicializa la lista de distancias si esta vacia
            if (nx, ny) not in self.distancesDirty:
                self.distancesDirty[(nx, ny)] = []
            for dirtyCell in self.dirtyCells:
                cx, cy = dirtyCell
                dirty_dist = self.calculate_distance((nx, ny), (cx, cy))
                self.distancesDirty[(nx, ny)].append(dirty_dist)  # Agrega la distancia

        # Recopila las distancias de sus vecinos a otros agentes
        for nx, ny in valid_neighbors:
            # Inicializa la lista de distancias si esta vacia
            if (nx, ny) not in self.distancesAgent:
                self.distancesAgent[(nx, ny)] = []      
            found_own_agent = False
            for agent in self.model.all_agents:
                ax, ay = agent.position
                if not found_own_agent and (x,y) == (ax,ay):  # Se ignora a si mismo
                    found_own_agent = True
                else:
                    agent_dist = self.calculate_distance((nx, ny), (ax, ay))
                    self.distancesAgent[(nx, ny)].append(agent_dist)  # Agrega la distancia


    def next(self):
        x, y = self.position
        """ Decide la próxima acción según lo que percibe """
        self.intention = "wait"  # Default action es esperar
        self.heatmap.clear()  # limpia el heatmap de la iteracion pasada

        # Calcula los valores del mapa de calor usando las distancias a las celdas sucias y otros agentes
        for (nx, ny) in self.distancesDirty.keys():  # Iterate los vecinos
            dirty_cell_heat = 0  # Inicializa el calor que aportan las celulas sucias
            agent_heat = 0  # Inicializa el calor de los agentes

            # Calcula el calor de las celdas sucias en los vecions usando la formula de calor
            if (nx, ny) in self.distancesDirty:
                for dirty_dist in self.distancesDirty[(nx, ny)]:
                    dirty_cell_heat += (1 / dirty_dist**2) * self.w1 + (1 / dirty_dist) * self.w2

            # Calcula el calor de las agentes en los vecinos usando la formula de calor
            if (nx, ny) in self.distancesAgent:
               for agent_dist in self.distancesAgent[(nx, ny)]:
                   agent_heat += (1 / agent_dist**2) * self.w3 + (1 / agent_dist) * self.w4

            # Combina el calor de las celdas sucias y los agentes
            self.heatmap[(nx, ny)] = dirty_cell_heat + agent_heat

        # Ordenamos los vecinos de mayor a menor calor
        sorted_neighbors = sorted(self.heatmap.items(), key=lambda item: item[1], reverse=True)

        # Asignamos probabilidades a los vecinos con mas calor
        neighbors = [neighbor for neighbor, _ in sorted_neighbors[:4]]  # Top 4 vecinos
        probabilities = [40, 30, 20, 10][:len(neighbors)]   # Probabilidades de los top 4 vecinos

        # Escogemos usando las probabilidades
        self.new_position = self.model.random.choices(neighbors, probabilities, k=1)[0]
        if self.model.is_dirty[x][y]:  
            self.intention = "clean"
        else:
            self.intention = "move"


    def action(self):
        """ Ejecuta la accion """
        x, y = self.position
        
        self.state_utility = 0

        if self.intention == "clean":
            self.model.clean_cell(x, y)
            self.state_utility = 1 # La utilidad de estado es 1 cuando limpia

        elif self.intention == "move":
            self.position = self.new_position
        self.run_utility = self.utility() 

    def move(self):
        if not self.model.is_board_clean:
            self.model.total_moves += 1
        
        self.see() 
        self.next()  
        self.action()  


In [ ]:
class OscarAgent(ap.Agent):
    """ Idea: Dar mayor probabilidad de moverse a celdas que estén más lejos de los demás agentes. Asimismo, considerar celda sucia más cercana. """
    def setup(self):
        self.position = self.p.initial_position  # Posición actual del agente (x, y)
        self.intention = None  # Intención actual (clean o move)
        self.new_position = None  # Objetivo al moverse ([-1 | 0 | 1], [-1 | 0 | 1])
        self.is_dirty = False
        self.cleaned_cells = 0
        self.initial_dirty_cells = self.model.dirty_cells
        self.closest_factor = 0.5
        
    def see(self):
        """Considerar celdas sin limpiar y otros agentes."""
        if self.model.is_dirty[self.position[0]][self.position[1]]:
            self.is_dirty = True
        else:
            self.is_dirty = False
            self.dirty_cells =  []
            
            for i in range(len(self.model.is_dirty)):
                for j in range(len(self.model.is_dirty[i])):
                    if self.model.is_dirty[i][j]:
                        self.dirty_cells.append((i,j))
            
            self.agent_locations = [agent.position for agent in self.model.all_agents]

    def point_distance(self, p1, p2):
        return abs(p1[0] - p2[0]) + abs(p1[1] - p2[1])
    # O( celdas sucias * agentes)
    def next(self):
        if self.is_dirty:
            self.intention = "clean"
        else:
            self.intention = "move"
            self.distances = [0] * len(self.dirty_cells)
            closest_index = 0
            self.total_distance = 0
            ignored_self = False
            
            for i in range(len(self.dirty_cells)):
                if self.point_distance(self.position, self.dirty_cells[i]) < self.point_distance(self.position, self.dirty_cells[closest_index]):
                    closest_index = i

                for agent in self.agent_locations:
                    if self.position[0] == agent[0] and self.position[1] == agent[1] and not ignored_self:
                        # No considerar la posicion actual del agente, para evitar que todos los agentes tengan las mismas probabilidades
                        ignored_self = True
                    else:
                        distance = abs(self.dirty_cells[i][0] - agent[0] + self.dirty_cells[i][1] - agent[1])
                        
                        self.distances[i] += distance
                        self.total_distance += distance
            
            probabilities = [0] * len(self.dirty_cells)
            
            # Evitar division por cero
            if self.total_distance == 0:
                self.total_distance = 1
                
            for i in range(len(self.dirty_cells)):
                probabilities[i] = self.distances[i] / self.total_distance
                
            # Considerar la celda sucia más cercana
            probabilities = [p * (1 - self.closest_factor) for p in probabilities]
            
            probabilities[closest_index] += self.closest_factor
            
            for i in range(len(self.dirty_cells)):
                if self.dirty_cells[i][0] == self.position[0] and self.dirty_cells[i][1] == self.position[1]:
                    probabilities[i] += self.closest_factor
            
            # Normalizar probabilidades en caso necesario
            if not np.isclose(sum(probabilities), 1.0):
                normalization_factor = sum(probabilities)
                probabilities = [p / normalization_factor for p in probabilities]
            
            
            
            # Elegir la celda objetivo considerando las probabilidades
            try:
                target_position_index = self.model.nprandom.choice([*range(len(self.dirty_cells))], p=probabilities)
            except ValueError:
                # Si hay algun error, elegir una celda al azar
                target_position_index = self.model.nprandom.choice([*range(len(self.dirty_cells))])
                
            target_position = self.dirty_cells[target_position_index]
            # Obtener movimiento necesario para llegar a la celda objetivo
            x = np.sign(target_position[0] - self.position[0])
            y = np.sign(target_position[1] - self.position[1])
            
            self.new_position = (x, y)
                
    def action(self):
        """ Ejecuta la accion """
        x, y = self.position
        if self.intention == "clean":
            self.model.clean_cell(x, y)
        elif self.intention == "move":
            dx, dy = self.new_position
            self.position = (x + dx, y + dy)
    
    def utility(self):
        return self.cleaned_cells / self.initial_dirty_cells * 100
            
    def move(self):
        if not self.model.is_board_clean:
            self.model.total_moves += 1     
            self.see()
            self.next()
            self.action()


In [ ]:
class PepeAgent(ap.Agent):
    '''Agente con patrones de movimiento en espiral y hacia celdas sucias.'''
    def setup(self):
        self.position = (0, 0) 
        self.intention = None
        self.new_position = None
        self.is_dirty = False

        #Control del espiral
        self.phase = 0  # 0=abajo, 1=derecha, 2=arriba, 3=izquierda
        self.top = 0
        self.bottom = len(self.model.is_dirty) - 1
        self.left = 0
        self.right = len(self.model.is_dirty[0]) - 1

    def see(self):
        x, y = self.position
        self.is_dirty = self.model.is_dirty[x][y]

    def select_pattern(self):
        #Define movimiento posibles
        patterns = ["spiral", "dirty_cells"]
        return self.model.nprandom.choice(patterns)

    def next(self):
        self.intention = "wait"
        if self.is_dirty:
            self.intention = "clean"
            return
        pattern = self.select_pattern()
        
        if pattern == "spiral":
            self.spiral_movement()
        elif pattern == "dirty_cells":
            self.move_towards_dirty_cell()


    # Movimientos en espiral y celda sucia más cercana 
    def spiral_movement(self):
        x, y = self.position
        moved = False

        if self.phase == 0:  # abajo
            if y < self.bottom:
                self.new_position = (x, y + 1)
                moved = True
            else:
                self.phase = 1
                self.left += 1

        if self.phase == 1:  #derecha
            if x < self.right:
                self.new_position = (x + 1, y)
                moved = True
            else:
                self.phase = 2
                self.bottom -= 1

        if self.phase == 2:  # arriba
            if y > self.top:
                self.new_position = (x, y - 1)
                moved = True
            else:
                self.phase = 3
                self.right -= 1

        if self.phase == 3:  # izquierda
            if x > self.left:
                self.new_position = (x - 1, y)
                moved = True
            else:
                self.phase = 0
                self.top += 1

        if moved and self.model.is_valid_position(self.new_position):
            self.intention = "move"


    """Funciones para el movimiento a la celda sucia más cercana"""
    def move_towards_dirty_cell(self):
        x, y = self.position
        dirty_cells = [
            (i, j) for i in range(len(self.model.is_dirty))
            for j in range(len(self.model.is_dirty[0]))
            if self.model.is_dirty[i][j]
        ]
        if dirty_cells:
            target_cell = min(dirty_cells, key=lambda cell: self.point_distance(cell, (x, y)))
            dx = np.sign(target_cell[0] - x)
            dy = np.sign(target_cell[1] - y)
            self.new_position = (x + dx, y + dy)
            self.intention = "move"

    def point_distance(self, p1, p2):
        return abs(p1[0] - p2[0]) + abs(p1[1] - p2[1])




    def action(self):
        x, y = self.position
        if self.intention == "clean":
            self.model.clean_cell(x, y)
        elif self.intention == "move":
            self.position = self.new_position

    def move(self):
        if not self.model.is_board_clean:
            self.model.total_moves += 1
        self.see()
        self.next()
        self.action()

In [ ]:
class HectorAgent(ap.Agent):
    """Estrategia adaptiva de cuadrantes"""
    
    def setup(self):
        self.position = (0,0)
        self.intention = None
        self.new_position = None
        self.is_dirty = False
        self.quadrant = None
        self.visited = set([(0,0)])
        self.stuck_counter = 0
        
    def _assign_initial_quadrant(self):
        """Asigna un cuadrante basado en la posicion del agente"""
        n, m = self.model.p.n, self.model.p.m
        mid_x, mid_y = n//2, m//2
        
        agent_idx = list(self.model.all_agents).index(self)
        total_agents = len(self.model.all_agents)
        
        quadrants = [
            [(0, 0), (mid_x, mid_y)],   # izq-sup
            [(mid_x, 0), (n, mid_y)],   # der-sup
            [(0, mid_y), (mid_x, m)],   # izq-inf
            [(mid_x, mid_y), (n, m)]    # der-inf
        ]
        
        return quadrants[agent_idx % 4]
    
    def _get_agent_density_map(self):
        """Genera un mapa de calor de las posiciones de otros agentes"""
        density = {}
        for agent in self.model.all_agents:
            if agent != self:
                pos = agent.position
                density[pos] = density.get(pos, 0) + 1
        return density
    
    def _score_position(self, pos):
        """Califica una posicion basado en ciertos factores"""
        if not self.model.is_valid_position(pos):
            return float('-inf')
            
        x, y = pos
        score = 0
        
        # Preferir celdas no visitadas
        if pos not in self.visited:
            score += 10
            
        # Preferir celdas en el cuadrante
        (qx1, qy1), (qx2, qy2) = self.quadrant
        if qx1 <= x < qx2 and qy1 <= y < qy2:
            score += 5
            
        # Evitar agentes
        density_map = self._get_agent_density_map()
        score -= density_map.get(pos, 0) * 3
        
        # Preferir celdas sucias
        if self.model.is_dirty[x][y]:
            score += 15
            
        return score
    
    def _get_best_move(self):
        """Determina el mejor movimiento basado en las calificaciones"""
        x, y = self.position
        possible_moves = self.model.get_neighbors(x, y)
        
        scored_moves = [
            (self._score_position(pos), pos)
            for pos in possible_moves
            if self.model.is_valid_position(pos)
        ]
        
        # Si falla varias veces expandir la busqueda
        if self.stuck_counter > 3:
            self.stuck_counter = 0
            valid_moves = [pos for _, pos in scored_moves if self.model.is_valid_position(pos)]
            return self.model.random.choice(valid_moves) if valid_moves else self.position
            
        if not scored_moves:
            self.stuck_counter += 1
            return self.position
            
        return max(scored_moves)[1]
    
    def see(self):
        if self.quadrant is None:
            self.quadrant = self._assign_initial_quadrant()
             
        x, y = self.position
        self.is_dirty = self.model.is_dirty[x][y]
        self.neighbors = self.model.get_neighbors(x, y)
    
    def next(self):
        self.intention = "wait"
        if self.is_dirty:
            self.intention = "clean"
        else:
            self.new_position = self._get_best_move()
            if self.new_position != self.position:
                self.intention = "move"
                
        self.visited.add(self.position)
    
    def action(self):
        x, y = self.position
        if self.intention == "clean":
            self.model.clean_cell(x, y)
        elif self.intention == "move":
            self.position = self.new_position
            
    def move(self):
        if not self.model.is_board_clean:
            self.model.total_moves += 1
        self.see()
        self.next()
        self.action()

In [ ]:
class CleaningModel(ap.Model):
    """ Modelo de limpieza de tablero """
    
    def generateDirtyCells(self):
        """
        Genera una matriz n x m donde un porcentaje de las celdas están marcadas como sucias (1) y el resto limpias (0).
        devuelve: Matriz n x m con celdas sucias y limpias
        """
        totalCells = self.p.n * self.p.m
    
        dirtyCellsCount = math.ceil(totalCells * (self.p.percentage_dirty / 100))

        # Inicializamos la matriz con ceros
        isDirty = [[0 for _ in range(self.p.m)] for _ in range(self.p.n)]

        # Generamos las posiciones de las celdas sucias de manera aleatoria
        allPositions = [(i, j) for i in range(self.p.n) for j in range(self.p.m)]
        dirtyPositions = self.random.sample(allPositions, dirtyCellsCount)

        # Marcamos las celdas seleccionadas como sucias
        for i, j in dirtyPositions:
            isDirty[i][j] = 1

        return isDirty
    
    def is_valid_position(self, position):
        """ Verifica si una posición es valida dentro de nuestro tablero """
        x, y = position
        return 0 <= x < self.p.n and 0 <= y < self.p.m
    
    def get_neighbors(self, x, y):
        """ regresa lista con todas las celdas vecinas de la posicion x, y"""
        neighbors = []
        
        for dx in range(-1, 2):  # Iterata -1, 0, 1
            for dy in range(-1, 2):  # Itera -1, 0, 1
                if not (dx == 0 and dy == 0):  # Saltamos la celda actual
                    neighbors.append((x + dx, y + dy)) 
    
        return neighbors
    
    def clean_cell(self, x, y): 
        if self.is_dirty[x][y] == 1:
            self.is_dirty[x][y] = 0 
            self.cleaned_cells += 1

        if not(self.is_board_clean) and self.cleaned_cells == self.dirty_cells:
            self.finish_time = self.t
            self.is_board_clean = True

    
    def setup(self):
        
        self.dirty_cells = math.ceil(self.p.n * self.p.m * (self.p.percentage_dirty / 100))
        self.is_dirty = self.generateDirtyCells()
        
        self.random_agents = ap.AgentList(self, self.p.random_agents, RandomAgent)
        self.sergio_agents = ap.AgentList(self, self.p.sergio_agents, SergioAgent)
        self.rodrigo_agents = ap.AgentList(self, self.p.rodrigo_agents, RodrigoAgent)
        self.oscar_agents = ap.AgentList(self, self.p.oscar_agents, OscarAgent)
        self.pepe_agents = ap.AgentList(self, self.p.pepe_agents, PepeAgent)
        self.hector_agents = ap.AgentList(self, self.p.hector_agents, HectorAgent)
        

        # self.all_agents = self.sergio_agents + self.rodrigo_agents + self.oscar_agents + self.pepe_agents + self.hector_agents
        self.all_agents = self.random_agents + self.sergio_agents + self.rodrigo_agents + self.oscar_agents + self.pepe_agents + self.hector_agents
        # Genera matriz donde isDirty[i][j] == 1 si la celda en la fila i y columna j esta sucia
        
        # Inicializamos la matriz con ceros
        agentCountMatrix = [[0 for _ in range(self.p.m)] for _ in range(self.p.n)]
        agentCountMatrix[0][0] = len(self.all_agents)
        
        # Variables para generar estadisticas
        self.total_moves = 0
        self.cleaned_cells = 0
        self.is_board_clean = True if self.p.percentage_dirty == 0 else False # Es falso a menos que no haya celdas sucias
        self.finish_time = self.p.steps + 1 # Por default no termina en tiempo


    def step(self):
        self.all_agents.move()

    def update(self):
        """
        self.record('Gini Coefficient (all_agents)', gini(self.all_agents.wealth))
    """

    def end(self):
        time_taken = self.finish_time
        cleaned_percentage = (self.cleaned_cells / (self.dirty_cells)) * 100
        clean_percentage = ((self.p.n * self.p.m - self.dirty_cells + self.cleaned_cells) / (self.p.n * self.p.m)) * 100
        
        
        # print()
        # if not self.is_board_clean:
        #     print("La limpieza no acabo, se llego al tiempo maximo")
        # else:
        #     print(f"Tiempo de limpieza: {time_taken} pasos")
            
        # print(f"Porcentaje de celdas limpiadas: {cleaned_percentage:.2f}%")
        # print(f"Porcentaje de celdas limpias: {clean_percentage:.2f}%")
        # print(f"Total de movimientos: {self.total_moves}")
        
        self.report('total_moves', self.total_moves)
        self.report('clean_percentage', clean_percentage)
        self.report('cleaned_percentage', cleaned_percentage)
        self.report('time_taken', self.finish_time)
        self.report('percentage_time_taken', self.finish_time / self.p.steps)

In [ ]:
def multirun_experiment():
    # ciudado! Se usara 2 ** simulations_log_2. No hagas este valor muy alto
    simulations_log_2 = 2
    iterations_per_sample = 2 
    parameters = {
        'random_agents': 0,
        'sergio_agents': 0,
        'rodrigo_agents': 0,
        'oscar_agents': 0,
        'pepe_agents': 0,
        'hector_agents': 10,
        'steps': ap.IntRange(10, 20*20), # From very low, to a very high value
        'n': ap.IntRange(1, 20),
        'm': ap.IntRange(1, 20),
        'percentage_dirty': ap.IntRange(1, 100),
        'initial_position': (0, 0),
        'seed': 22,
    }

    agent_types = [ 
                    'sergio_agents',
                    'rodrigo_agents',
                    'oscar_agents',
                    'pepe_agents',
                    'hector_agents',
                    ]

    runs = {}
    for agent in agent_types: 
        # if agent != 'sergio_agents':  # Uncomment to test only your agent
        #     continue
        
        # Asign valid values to current agent only
        for other_agent in agent_types:
            if agent == other_agent:
                parameters[other_agent] = ap.IntRange(1, 50)
            else: 
                parameters[other_agent] = 0
                
        sample = ap.Sample(
            parameters,
            n= 2 ** simulations_log_2,
            method='saltelli',
            calc_second_order=False
        )
        exp = ap.Experiment(CleaningModel, sample, iterations=iterations_per_sample, record=True)
        results = exp.run()
        reporters_df = results.reporters
        
        curr_runs = {
            'A': 0,
            'B': 0,
            'C': 0,
            'D': 0,
            'not_finished': 0,
            'total': 0,
        }
        
        curr_runs['total'] = len(reporters_df)
        curr_runs['A'] = (reporters_df['percentage_time_taken'] <= 0.25).sum() / curr_runs['total']
        curr_runs['B'] = (reporters_df['percentage_time_taken'] <= 0.50).sum() / curr_runs['total']
        curr_runs['C'] = (reporters_df['percentage_time_taken'] <= 0.75).sum() / curr_runs['total']
        curr_runs['D'] = (reporters_df['percentage_time_taken'] <= 1.00).sum() / curr_runs['total']

        curr_runs['not_finished_probability'] = (reporters_df['percentage_time_taken'] > 1.00).sum() / curr_runs['total']
        
        curr_runs['A'] = round(curr_runs['A'], 3)
        curr_runs['B'] = round(curr_runs['B'], 3)
        curr_runs['C'] = round(curr_runs['C'], 3)
        curr_runs['D'] = round(curr_runs['D'], 3)
        curr_runs['not_finished_probability'] = round(curr_runs['not_finished_probability'], 3)
        runs[agent] = curr_runs
        
    
    return runs
        

In [ ]:
def test_time_taken():
    # ciudado! Se usara 2 ** simulations_log_2. No hagas este valor muy alto
    simulations_log_2 = 2
    iterations_per_sample = 2 
    parameters = {
        'random_agents': 0,
        'sergio_agents': 0,
        'rodrigo_agents': 0,
        'oscar_agents': 0,
        'pepe_agents': 0,
        'hector_agents': 0,
        'steps': 400, 
        'n': 13,
        'm': 13,
        'percentage_dirty': 70,
        'initial_position': (0, 0),
        'seed': 22,
    }

    agent_types = [ 
                    'sergio_agents',
                    'rodrigo_agents',
                    'oscar_agents',
                    'pepe_agents',
                    'hector_agents',
                    ]

    agents_time_df = pd.DataFrame(columns=['time_taken', 'agent_amount', 'agent_type'])
    for agent in agent_types: 
        # if agent != 'sergio_agents':  # Uncomment to test only your agent
        #     continue
        
        # Asign valid values to current agent only
        for other_agent in agent_types:
            if agent == other_agent:
                parameters[other_agent] = ap.IntRange(1, 50)
            else: 
                parameters[other_agent] = 0
                
        sample = ap.Sample(
            parameters,
            n= 2 ** simulations_log_2,
            method='saltelli',
            calc_second_order=False
        )
        exp = ap.Experiment(CleaningModel, sample, iterations=iterations_per_sample, record=True)
        results = exp.run()
        
        # Update agents vs time_taken
        agent_df = results.parameters.sample
        agent_df = agent_df[[agent]]
        reporters_df : pd.DataFrame = results.reporters[['time_taken', 'percentage_time_taken', 'total_moves']]
        reporters_df = reporters_df[reporters_df['percentage_time_taken'] <= 1] # Filter only finished runs to clean
        
        reporters_df_time = reporters_df[['time_taken']]
        temp_agent_time_df = reporters_df_time.merge(agent_df, on='sample_id', how='left')
        temp_agent_time_df = temp_agent_time_df.reset_index()
        temp_agent_time_df = temp_agent_time_df[['time_taken', agent]]
        temp_agent_time_df.columns = ['time_taken', 'agent_amount'] 
        temp_agent_time_df['agent_type'] = agent
        agents_time_df = pd.concat([agents_time_df, temp_agent_time_df])
        
        
    
    sns.scatterplot(data=agents_time_df, x='agent_amount', y='time_taken', hue='agent_type', s=40)
    plt.title('Agent amount vs time taken to clean all')
    # Adjust Layout
    plt.tight_layout()
    plt.show()
        

In [ ]:
def test_mov_amount():
    # ciudado! Se usara 2 ** simulations_log_2. No hagas este valor muy alto
    simulations_log_2 = 2
    iterations_per_sample = 2 
    parameters = {
        'random_agents': 0,
        'sergio_agents': 0,
        'rodrigo_agents': 0,
        'oscar_agents': 0,
        'pepe_agents': 0,
        'hector_agents': 0,
        'steps': 400, 
        'n': 13,
        'm': 13,
        'percentage_dirty': 70,
        'initial_position': (0, 0),
        'seed': 22,
    }

    agent_types = [ 
                    'sergio_agents',
                    'rodrigo_agents',
                    'oscar_agents',
                    'pepe_agents',
                    'hector_agents',
                    ]

    agents_mov_df = pd.DataFrame(columns=['total_moves', 'agent_amount', 'agent_type'])
    for agent in agent_types: 
        # if agent != 'sergio_agents':  # Uncomment to test only your agent
        #     continue
        
        # Asign valid values to current agent only
        for other_agent in agent_types:
            if agent == other_agent:
                parameters[other_agent] = ap.IntRange(1, 50)
            else: 
                parameters[other_agent] = 0
                
        sample = ap.Sample(
            parameters,
            n= 2 ** simulations_log_2,
            method='saltelli',
            calc_second_order=False
        )
        exp = ap.Experiment(CleaningModel, sample, iterations=iterations_per_sample, record=True)
        results = exp.run()
        
        # Update agents vs total_movements
        agent_df = results.parameters.sample
        agent_df = agent_df[[agent]]
        reporters_df : pd.DataFrame = results.reporters[['time_taken', 'percentage_time_taken', 'total_moves']]
        reporters_df = reporters_df[reporters_df['percentage_time_taken'] <= 1] # Filter only finished runs to clean
        
        reporters_df_moves = reporters_df[['total_moves']]
        temp_agent_mov = reporters_df_moves.merge(agent_df, on='sample_id', how='left')
        temp_agent_mov = temp_agent_mov.reset_index()
        temp_agent_mov = temp_agent_mov[['total_moves', agent]]
        temp_agent_mov.columns = ['total_moves', 'agent_amount']
        temp_agent_mov['agent_type'] = agent
        agents_mov_df = pd.concat([agents_mov_df, temp_agent_mov])
        
    
    sns.scatterplot(data=agents_mov_df, x='agent_amount', y='total_moves', hue='agent_type', s=40)
    plt.title('Agent amount vs total movements done')

        

In [ ]:
def show_animation():
    parameters = {
        'random_agents': 0,
        'sergio_agents': 0,
        'rodrigo_agents': 0,
        'oscar_agents': 0,
        'pepe_agents': 10,
        'hector_agents': 0,
        'steps': 900,
        'n': 50,
        'm': 50,
        'percentage_dirty': 50,
        'initial_position': (0, 0),
        # 'seed': 22,
    }

    def animation_plot(model, ax):
        # White: Clean, Black: Dirty, Red: Agent
        color_dict = {0:'#FFFFFF', 1:'#000000', 2:'#FF0000'}
        ndarray_doubles = np.array(model.is_dirty, dtype=float)
        
        for agent in model.all_agents:
            x, y = agent.position
            ndarray_doubles[x][y] = 2
            
        ap.gridplot(ndarray_doubles, ax=ax, color_dict=color_dict, convert=True)
        ax.set_title(f"Cleaning model \n Time-step: {model.t}")
        
    fig, ax = plt.subplots()
    model = CleaningModel(parameters)
    animation = ap.animate(model, fig, ax, animation_plot)
    return animation
    

In [ ]:
# Grafica A, B, C, D
def plot_results(runs):
    categories = ['A', 'B', 'C', 'D']
    agents = list(runs.keys())
    data = {category: [runs[agent][category] for agent in agents] for category in categories}

    x = np.arange(len(agents))
    width = 0.2  

    fig, ax = plt.subplots(figsize=(10, 6))  
    for i, category in enumerate(categories):
        ax.bar(x + i * width, data[category], width, label=category)

    ax.set_xlabel('Agent Types')
    ax.set_ylabel('Proportion')
    ax.set_title('Comparison of Results by Agent Type')
    ax.set_xticks(x + width * (len(categories) - 1) / 2)
    ax.set_xticklabels([name.split('_')[0].capitalize() for name in agents], rotation=45, ha='right')  
    ax.legend()

    plt.tight_layout()  
    plt.show()


In [ ]:
options = {
    'show_animation': True,
    'multirun_experiment': True,
    'test_time_taken': True,
    'test_mov_amount': True,
}

In [ ]:
if options['show_animation']:
    animation = show_animation()
    display(
        HTML(animation.to_jshtml())
    )    

In [ ]:
if options['multirun_experiment']:
    results = multirun_experiment()
    plot_results(results)
    print(results)

In [ ]:
if options['test_time_taken']:
    test_time_taken()

In [ ]:
if options['test_mov_amount']:
    test_mov_amount()